In [1]:
import re

In [2]:
from tabulate import tabulate


platform = """
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""

tilted_platform = """
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
"""


def parse(platform: str) -> list[list[chr]]:
    return [list(l) for l in platform.strip().splitlines()]


def roll_stone_north(platform: list[list[chr]], stone_row: int, stone_col: int) -> None:
    row = stone_row - 1
    while row > -1 and platform[row][stone_col] == ".":
        row -= 1
    row += 1
    platform[stone_row][stone_col], platform[row][stone_col] = (
        platform[row][stone_col],
        platform[stone_row][stone_col],
    )


def slide_platform(platform: list[list[chr]]) -> list[list[chr]]:
    for row in range(1, len(platform)):
        for col in range(len(platform[0])):
            if platform[row][col] == "O":
                roll_stone_north(platform, row, col)
    return platform


assert slide_platform(parse(platform)) == parse(tilted_platform)


def count_rolingstones(row: list[chr]) -> int:
    return sum((1 for c in row if c == "O"), start=0)


def determine_load(platform: str) -> int:
    platform = slide_platform(parse(platform))
    n = len(platform)
    return sum((n - i) * count_rolingstones(r) for i, r in enumerate(platform))


assert determine_load(platform) == 136

In [3]:
with open("../input/day14.txt") as f:
    print(determine_load(f.read()))

105208


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>105208</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "<em>spin cycle</em>" attempts to do just that!</p>
<p>Each <em>cycle</em> tilts the platform four times so that the rounded rocks roll <em>north</em>, then <em>west</em>, then <em>south</em>, then <em>east</em>. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.</p>
<p>Here's what happens in the example above after each of the first few cycles:</p>
<pre><code>After 1 cycle:
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....

After 2 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O

After 3 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
</code></pre>

<p>This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the <em>total load</em> on the north support beams after <code>1000000000</code> cycles.</p>
<p>In the above example, after <code>1000000000</code> cycles, the total load on the north support beams is <code><em>64</em></code>.</p>
<p>Run the spin cycle for <code>1000000000</code> cycles. Afterward, <em>what is the total load on the north support beams?</em></p>
</article>

</main>


In [4]:
from collections import defaultdict
from itertools import product
from tabulate import tabulate


platform = """
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""

cycled_platform = """
After 1 cycle:
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....

After 2 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O

After 3 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
"""


def parse(platform: str) -> list[list[chr]]:
    return [list(l) for l in platform.strip().splitlines()]


def roll_stone_north(platform: list[list[chr]], stone_row: int, stone_col: int) -> None:
    row = stone_row - 1
    while row > -1 and platform[row][stone_col] == ".":
        row -= 1
    row += 1
    platform[stone_row][stone_col], platform[row][stone_col] = (
        platform[row][stone_col],
        platform[stone_row][stone_col],
    )


def roll_stone_east(platform: list[list[chr]], stone_row: int, stone_col: int) -> None:
    col = stone_col + 1
    while col < len(platform[0]) and platform[stone_row][col] == ".":
        col += 1
    col -= 1
    platform[stone_row][stone_col], platform[stone_row][col] = (
        platform[stone_row][col],
        platform[stone_row][stone_col],
    )


def roll_stone_south(platform: list[list[chr]], stone_row: int, stone_col: int) -> None:
    row = stone_row + 1
    while row < len(platform) and platform[row][stone_col] == ".":
        row += 1
    row -= 1
    platform[stone_row][stone_col], platform[row][stone_col] = (
        platform[row][stone_col],
        platform[stone_row][stone_col],
    )


def roll_stone_west(platform: list[list[chr]], stone_row: int, stone_col: int) -> None:
    col = stone_col - 1
    while col > -1 and platform[stone_row][col] == ".":
        col -= 1
    col += 1
    platform[stone_row][stone_col], platform[stone_row][col] = (
        platform[stone_row][col],
        platform[stone_row][stone_col],
    )


def cycle(platform: list[list[chr]]) -> None:
    roll_north(platform)
    roll_west(platform)
    roll_south(platform)
    roll_east(platform)


def roll_east(platform):
    for col, row in product(range(len(platform[0]) - 1, -1, -1), range(len(platform))):
        if platform[row][col] == "O":
            roll_stone_east(platform, row, col)


def roll_south(platform):
    for row, col in product(range(len(platform) - 1, -1, -1), range(len(platform[0]))):
        if platform[row][col] == "O":
            roll_stone_south(platform, row, col)


def roll_west(platform):
    for col, row in product(range(1, len(platform[0])), range(len(platform))):
        if platform[row][col] == "O":
            roll_stone_west(platform, row, col)


def roll_north(platform):
    for row, col in product(range(1, len(platform)), range(len(platform[0]))):
        if platform[row][col] == "O":
            roll_stone_north(platform, row, col)


def count_rolingstones(row: list[chr]) -> int:
    return sum((1 for c in row if c == "O"), start=0)


def determine_load_II(platform: str) -> int:
    platform = parse(platform)
    pf = tuple(tuple(r) for r in platform)
    cycles = {pf}
    platforms = [pf]

    for i in range(1_000_000_000):
        cycle(platform)
        pf = tuple(tuple(r) for r in platform)
        if pf in cycles:
            i += 1
            break
        cycles.add(pf)
        platforms.append(pf)

    first = platforms.index(pf)
    platform = platforms[(1_000_000_000 - first) % (i - first) + first]
    n = len(platform)
    return sum((n - i) * count_rolingstones(r) for i, r in enumerate(platform))


matrici = [l.split(":") for l in re.split(r"\n\s*\n", cycled_platform.strip())]
matrici = [parse(m) for _, m in matrici]
pf = parse(platform)

for i, m in enumerate(matrici, start=1):
    cycle(pf)
    assert pf == m

assert determine_load_II(platform) == 64
# determine_load_II(platform)

In [5]:
with open("../input/day14.txt") as f:
    print(determine_load_II(f.read()))

102943


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>102943</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2023">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="14/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Parabolic+Reflector+Dish%22+%2D+Day+14+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F14&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Parabolic+Reflector+Dish%22+%2D+Day+14+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F14';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
